In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [2]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN
4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN


In [5]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #464024
ppp_geonull

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,DELTA JUNCTION,AK,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,DELTA JUNCTION,AK,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,Anchorage,AK,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,HALIBUT COVE,AK,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,ANCHORAGE,AK,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972476,972476,972476,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,...,999 Other,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",NaN,NaN,NaN
972477,972477,972477,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,WY,...,999 Other,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",NaN,NaN,NaN
972478,972478,972478,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,WY,...,999 Other,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",NaN,NaN,NaN
972479,972479,972479,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,WY,...,999 Other,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",NaN,NaN,NaN


In [6]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
IN    16601
MO    16350
LA    13668
OR    13456
UT     9295
IA     8295
NV     8262
KS     8202
NE     5804
MS     5502
HI     4980
NM     4485
ID     4416
RI     3792
ND     3222
MT     3144
PR     2952
OK     2839
SD     2695
SC     2335
WY     2139
GU      443
VI      290
MP       82
NC       11
PA        8
AL        8
AK        5
AZ        4
NJ        3
TN        3
MA        3
IL        2
ME        2
AR        2
OH        1
CT        1
WV        1
GA        1
Name: BorrowerState, dtype: int64

In [6]:
ppp_togeoOK = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "OK"]
ppp_togeoOK #9996

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,FIPS_z,Lat,Long
712600,712600,712600,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,...,999 Other,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",NaN,NaN,NaN
712601,712601,712601,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",...,325 Chemicals,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",NaN,NaN,NaN
712602,712602,712602,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",...,999 Other,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",NaN,NaN,NaN
712603,712603,712603,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,...,999 Other,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",NaN,NaN,NaN
712604,712604,712604,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722591,722591,722591,722591,722591,8894878401,2/14/21,680,PPS,PPGLLC,1110 Cornwell Dr,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1110 Cornwell Dr, Yukon, OK",NaN,NaN,NaN
722592,722592,722592,722592,722592,8956648300,1/30/21,680,PPS,JS&J ENTERPRISES LLC,205 S Main St,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"205 S Main St, Sapulpa, OK",NaN,NaN,NaN
722593,722593,722593,722593,722593,9099848507,3/12/21,680,PPP,THE SEQUOYAH COUNTY WATER ASSOCIATION INC.,3520 W. Cherokee,...,999 Other,"2213 Water, Sewage and Other Systems",2213.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3520 W. Cherokee, Sallisaw, OK",NaN,NaN,NaN
722594,722594,722594,722594,722594,9183018301,1/30/21,680,PPS,WILKIE MFG LLC,2640 NW 2nd St,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2640 NW 2nd St, Oklahoma City, OK",NaN,NaN,NaN


In [7]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoOK_add = ppp_togeoOK[cols]
ppp_togeoOK_add

,LoanNumber,full_add
712600,1008918710,"15615 E 590 Rd, Inola, OK"
712601,1994657208,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK"
712602,2427157100,"4500 S. 129th E Ave, Suite 150, TULSA, OK"
712603,6097937201,"390 NW 36TH ST, OKLAHOMA CITY, OK"
712604,6262678403,"2309 N Lelia St, Guymon, OK"
...,...,...
722591,8894878401,"1110 Cornwell Dr, Yukon, OK"
722592,8956648300,"205 S Main St, Sapulpa, OK"
722593,9099848507,"3520 W. Cherokee, Sallisaw, OK"
722594,9183018301,"2640 NW 2nd St, Oklahoma City, OK"


In [8]:
add = ppp_togeoOK["full_add"].astype(str).to_list()

In [13]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text)
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('40')] #OK
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK is Not Found
2309 N Lelia St, Guymon, OK is Not Found
9600 Broadway Extension, Oklahoma City, OK is Not Found
5801 N BROADWAY EXT STE 210, OKLAHOMA CITY, OK is Not Found
100 Enterprise B;vd, PAULS VALLEY, OK is Not Found
401 S Boston Ave, Ste #1200 0.0, Tulsa, OK is Not Found
10205 E 161ST ST, TULSA, OK is Not Found
301 James Dean Dr, NORMAN, OK is Not Found
301 David L Boren Blvd Ste 2000, NORMAN, OK is Not Found
12558 Smiley Rd, MADILL, OK is Not Found
1200 S G St, MUSKOGEE, OK is Not Found
6707 N. Interstate Drive, NORMAN, OK is Not Found
3378 West Highway 117, Sapulpa, OK is Not Found
69300 Nee Rd, QUAPAW, OK is Not Found
1041 1ST ST, MADILL, OK is Not Found
605 SW B Avenue, Suite 2, LAWTON, OK is Not Found
4501 South 86th East Ave. 0.0, Tulsa, OK is Not Found
6729 NW 39th Expressway, BETHANY, OK is Not Found
324 N robinson sve Suite 100, OKLAHOMA CITY, OK is Not Found
4492 HUNT ST, PRYOR, OK is Not Found
6475 S Yale Ste 301, TULSA,

1111 N Lee Ave Ste 334, Oklahoma City, OK is Not Found
1815 SE BISON ROAD, BARTLESVILLE, OK is Not Found
ADMIRAL PL, TULSA, OK is Not Found
69300 Nee Rd, QUAPAW, OK is Not Found
177855 NCR 3030 Road, Velma, OK is Not Found
4201 S RADIO RD, EL RENO, OK is Not Found
100 WOODBRIAR, NOBLE, OK is Not Found
8526 S Peoria Ave,, TULSA, OK is Not Found
11277 N. HIGHWAY 99, SEMINOLE, OK is Not Found
2319 west main st, PRAGUE, OK is Not Found
4420 Anderson Road, Woodward, OK is Not Found
16601 W SOUTH AVE, TONKAWA, OK is Not Found
400 PO BOX 400, DUNCAN, OK is Not Found
4320 Adams Rd, Norman, OK is Not Found
1700 Crystal Lk, Blanchard, OK is Not Found
1700 CRYSTAL LAKE DR, BLANCHARD, OK is Not Found
5699 S 59th West Ave Ste 200, Tulsa, OK is Not Found
PO BOX 129, ANTLERS, OK is Not Found
1601 NW Expressway Ste 1450, Oklahoma City, OK is Not Found
1601 NW Expwy St, Suite 1450, OKLAHOMA CITY, OK is Not Found
650 PO BOX, PAULS VALLEY, OK is Not Found
48 N Cooley Dr, Oklahoma City, OK is Not Found
45

7604 Highway 132, Drummond, OK is Not Found
1028 WCHERRY AVE, ENID, OK is Not Found
14301 CALIBER DR STE 300, OKLAHOMA CITY, OK is Not Found
2201 N 170th East Ave, Tulsa, OK is Not Found
3200 Quail Springs Pkwy, Ste 200, OKLAHOMA CITY, OK is Not Found
6420 S 39th Ave, TULSA, OK is Not Found
7613 N Classen Blvd, Oklahoma City, OK is Not Found
5054 Western Ave, WOODWARD, OK is Not Found
4939 PO BOX, TULSA, OK is Not Found
PO Box 197, DOVER, OK is Not Found
97 State Highway 123, BARNSDALL, OK is Not Found
3400 S. Sheridan Rd., NORMAN, OK is Not Found
8700 NW Expressway, OKLAHOMA CITY, OK is Not Found
4620 B and B Dr, Woodward, OK is Not Found
2915 N Highway 99, Stroud, OK is Not Found
2915 N HIGHWAY 99, STROUD, OK is Not Found
4620 B AND B DR, WOODWARD, OK is Not Found
97 State Hwy 123, Barnsdall, OK is Not Found
2424 East 21st Street, STE 150, TULSA, OK is Not Found
510 NORTH BROADWAY, DAVENPORT, OK is Not Found
183922 E County Road 60, Arnett, OK is Not Found
183922 E COUNTY ROAD 60, AR

12755 S 705 RD, WYANDOTTE, OK is Not Found
WATERWOOD PKWY, EDMOND, OK is Not Found
2501 E INDEPENDENCE ST STE 100, SHAWNEE, OK is Not Found
4333 S 86th E Ave, TULSA, OK is Not Found
7 NE 6th St Ste 30, Oklahoma City, OK is Not Found
7 NE 6TH ST STE 210, OKLAHOMA CITY, OK is Not Found
8800 NW EXPRESSWAY, OKLAHOMA CITY, OK is Not Found
6968 E 610 RD, LOCUST GROVE, OK is Not Found
2401 NW 23RD ST Suite 39A, OKLAHOMA CITY, OK is Not Found
3126 S. BOULEVARD # 143, EDMOND, OK is Not Found
9800 S 219th East Ave, Broken Arrow, OK is Not Found
9800 S 219th E Ave, BROKEN ARROW, OK is Not Found
610 West Truck Bypass, DRUMRIGHT, OK is Not Found
11063D S Memorial Dr # 343, Tulsa, OK is Not Found
3800 ADAMS RD, BARTLESVILLE, OK is Not Found
1108 PO BOX, EL RENO, OK is Not Found
1209 N 142nd E Ave, Tulsa, OK is Not Found
17450 PO Box 17450, OKLAHOMA CITY, OK is Not Found
6465 South Yale Avenue, Suite 507, Tulsa, OK is Not Found
7190 BLUEGRASS LN, KINGSTON, OK is Not Found
400 INDUSTRIAL PARK A., LIND

...

650 W Commerce St, Clinton, OK is Not Found
1800 West Carrier Road, Enid, OK is Not Found
905 Beall Rd, KINGFISHER, OK is Not Found
420 Lonnie Abbott Blvd, ADA, OK is Not Found
15609 E 840 Rd, Kingfisher, OK is Not Found
1948 PLUNKETVILLE ROAD, WATSON, OK is Not Found
7100 NW Classen Suite 100, OKLAHOMA CITY, OK is Not Found
4425 Eats 31st #210, TULSA, OK is Not Found
2833 Conference Dr, Edmond, OK is Not Found
1637 CRYSTAL LAKE DR, BLANCHARD, OK is Not Found
Po box 1741, STILLWATER, OK is Not Found
1995 N Public St Ste 1, Tonkawa, OK is Not Found
1995 N Public Street, TONKAWA, OK is Not Found
1125 South Industrial Rd, CHANDLER, OK is Not Found
10314 E 138th E Ave, OWASSO, OK is Not Found
100TH ST, OWASSO, OK is Not Found
12802 N 103RD EAST AVE, COLLINSVILLE, OK is Not Found
2300 S BROADWAY AVE, POTEAU, OK is Not Found
4 NE 10TH ST #428, OKLAHOMA CITY, OK is Not Found
5833 NW Expressway, Warr Acres, OK is Not Found
11600 Broadway Extension Suite 150, Oklahoma City, OK is Not Found
2546

100 East California, C100, OKLAHOMA CITY, OK is Not Found
5600 N MAY AVE, STE 330, OKLAHOMA CITY, OK is Not Found
101 First St, Lahoma, OK is Not Found
12583 S Highway 99, Seminole, OK is Not Found
210 Park Ave Ste 3100, Oklahoma City, OK is Not Found
210 PARK AVE STE 3100, OKLAHOMA CITY, OK is Not Found
840 Research Parkway, Suite 250, OKLAHOMA CITY, OK is Not Found
12910 Broadway Ext, Oklahoma City, OK is Not Found
3158 SOUTH 108TH EAST AVE SUITE 274, TULSA, OK is Not Found
4501 S Mill, PRYOR, OK is Not Found
3449 W 60th St N, Porter, OK is Not Found
909 W I 35 FRONTAGE RD, EDMOND, OK is Not Found
2401 NW 23RD ST STE 80, OKLAHOMA CITY, OK is Not Found
3509 French Park Dr Ste D, EDMOND, OK is Not Found
PO BOX 21157, OKLAHOMA CITY, OK is Not Found
2015 West Wrangler Blvd, SEMINOLE, OK is Not Found
10708 HIGHWAY 7 WEST, RATLIFF CITY, OK is Not Found
P.O. Box 779, OKLAHOMA CITY, OK is Not Found
1215 N MAIN ST, BRISTOW, OK is Not Found
2773 STATE HIGHWAY 76, WILSON, OK is Not Found
900 NW

25905 E Admiral Pl, Catoosa, OK is Not Found
1931 N 170TH E AVE, TULSA, OK is Not Found
1567 E Maple Rd, Purcell, OK is Not Found
17297 E 715 Rd, Dover, OK is Not Found
107 North Boulder Ave, Suite A, TULSA, OK is Not Found
PO BOX 0257, NORTH MIAMI, OK is Not Found
179791 2820 RD, COMANCHE, OK is Not Found
PO Box 875, HEALDTON, OK is Not Found
3540 S Boulevard Ste 225, Edmond, OK is Not Found
9913 S May Ave Ste C, Oklahoma City, OK is Not Found
4137 S Harvard Ste F, TULSA, OK is Not Found
731 KERR RESEARCH DR, ADA, OK is Not Found
2601 NW EXPRESSWAY Suite 801W, OKLAHOMA CITY, OK is Not Found
279 PO BOX, FITTSTOWN, OK is Not Found
3401 NW 63rd Ste 555, OKLAHOMA CITY, OK is Not Found
200 RUSSELL M PERRY AVE, OKLAHOMA CITY, OK is Not Found
2245 N Air Depot Blvd N/A, Oklahoma City, OK is Not Found
3588 TOM SMITH RD, DURANT, OK is Not Found
1200 NW 63RD ST SUITE 5000, OKLAHOMA CITY, OK is Not Found
105 ROBINSON PL, ELK CITY, OK is Not Found
2216 Bridgeview Blvd, Edmond, OK is Not Found
PO B

1705 N 166th East Ave, Tulsa, OK is Not Found
1107 FELIX PL, OKLAHOMA CITY, OK is Not Found
3415 I 35 SERVICE RD, OKLAHOMA CITY, OK is Not Found
3540 S. BOULEVARD SUITE 300, EDMOND, OK is Not Found
14000 Quail Springs Pkwy Ste 1200, Oklahoma City, OK is Not Found
1415 S HIGHWAY 183, CLINTON, OK is Not Found
9520 N May Ave Ste 301, OKLAHOMA CITY, OK is Not Found
9520 N May Ave Ste 301, Oklahoma City, OK is Not Found
386 PO Box 386, JONES, OK is Not Found
2012TH AVE NW, ARDMORE, OK is Not Found
115 Piedmont Rd N, Piedmont, OK is Not Found
3415 S I 35 Service Rd, Oklahoma City, OK is Not Found
PO BOX 806, BEAVER, OK is Not Found
3701 S Merritt Rd, Elk City, OK is Not Found
6401 SE 74st, OKLAHOMA CITY, OK is Not Found
11016 E 81st, TULSA, OK is Not Found
16052 S BROADWAY, TULSA, OK is Not Found
700 N Wilson Rd, Weatherford, OK is Not Found
RR1 box 8, Knowles, OK is Not Found
304 N. Meridian, Suite 18, OKLAHOMA CITY, OK is Not Found
11645 DAKOTA, TONKAWA, OK is Not Found
6730 E 490, SALINA,

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
print(len(lat))
print(len(long))
print(len(geo_id))

10014
9996
9996


In [ ]:
#pd.set_option('display.max_rows', None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df

In [ ]:
test_loc = lat_df
test_loc = test_loc.sort_values(["Lat"], ascending=[True], ignore_index=False)
test_loc

In [61]:
lat_df.loc[lat_df.index[8730], "Lat"] = "35.48235"

In [ ]:
lat_drop = lat_df.drop(index=[7169,3081,8986,9628,1102,1275,2843,5548,5545,7276,3576,8034,8036,8493,5149,7330,4985,4987])
lat_drop = lat_drop.reset_index()
lat_drop

In [63]:
lat_list = lat_drop["Lat"].to_list()

In [ ]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat_list, "Long":long, "FIPS_z": geo_id})
result_df 

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [ ]:
#ppp_togeoOK_merge = ppp_togeoOK.merge(result_df, on="full_add", how ="left")
ppp_togeoOK_merge["Lat"] = ppp_togeoOK_merge["Lat_y"]
ppp_togeoOK_merge["Long"] = ppp_togeoOK_merge["Long_y"]
ppp_togeoOK_merge["FIPS_z"] = ppp_togeoOK_merge["FIPS_z_y"]
ppp_togeoOK_merge = ppp_togeoOK_merge.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_x", "FIPS_z_y"], axis=1)
ppp_togeoOK_merge


In [68]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoOK_parsed = ppp_togeoOK_merge[cols]
ppp_togeoOK_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
0,1008918710,401310505011056,36.1625,-95.554756
1,1008918710,401310505011056,36.1625,-95.554756
2,1008918710,401310505011056,36.1625,-95.554756
3,1994657208,None,None,None
4,2427157100,401430059001003,36.217976,-95.8332


In [69]:
ppp_geo = ppp_geo.merge(ppp_togeoOK_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,FIPS_z_x,Lat_x,Long_x,FIPS_z_y,Lat_y,Long_y
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",NaN,33.853682,-80.369867,NaN,NaN,NaN
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",NaN,32.942332,-80.066165,NaN,NaN,NaN
2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",NaN,47.636067,-122.302559,NaN,NaN,NaN
3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",NaN,34.224719,-117.372550,NaN,NaN,NaN
4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",NaN,42.521828,-82.913680,NaN,NaN,NaN


In [70]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [76]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo_cens.csv")